# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 12
REFERENCE_PANEL = "1000G"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: 1000G"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/gene_corrs-chr12.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite/gene_corrs-chr12.pkl"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  # eigenvalues <- eigen(data)$values
  # nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  cholStatus <- try(u <- chol(data), silent = FALSE)
  cholError <- ifelse(class(cholStatus) == "try-error", TRUE, FALSE)

  if (cholError) {
    return(FALSE)
  } else {
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 389 389

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000120645,ENSG00000111181,ENSG00000171840,ENSG00000060237,ENSG00000002016,ENSG00000082805,ENSG00000111186,ENSG00000006831,ENSG00000151062,ENSG00000151065
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000120645,1.000000000,0.082722583,0.01644917,0.01630049,0.007930999,0.01262378,0.009412586,0.013591059,0.01187203,0.011276547
ENSG00000111181,0.082722583,1.000000000,0.01666816,0.01604548,0.004979664,0.01788684,0.015020390,0.013956040,0.01272389,0.011573915
ENSG00000171840,0.016449170,0.016668163,1.00000000,0.13973255,0.010345402,0.02052182,0.016926274,0.013257508,0.01582615,0.009954466
ENSG00000060237,0.016300489,0.016045478,0.13973255,1.00000000,0.208445670,0.02714353,0.015065353,0.010928490,0.01447965,0.010851103
ENSG00000002016,0.007930999,0.004979664,0.01034540,0.20844567,1.000000000,0.02590133,0.015070122,0.007501334,0.01173014,0.006956180
ENSG00000082805,0.012623780,0.017886843,0.02052182,0.02714353,0.025901330,1.00000000,0.034333961,0.013125431,0.02150922,0.020115600


# Check positive definiteness

In [17]:
orig_matrix_is_positive_definite <- is_positive_definite(gene_corrs)

In [18]:
if (orig_matrix_is_positive_definite) {
  IRdisplay::display("Matrix is already positive definite!")
  file.copy(INPUT_FILE, OUTPUT_FILE)
} else {
  IRdisplay::display("We need to correct the data and make the matrix positive definite")

  eigenvalues <- eigen(gene_corrs)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
  IRdisplay::display(length(nonpositive_eigenvalues))
  IRdisplay::display(nonpositive_eigenvalues)

  # Make matrix positive definite
  gene_corrs_corrected <- CorrectCM(gene_corrs, 1e-10)
  dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)
  dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)
  gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

  # check if new matrix is really positive definite
  matrix_is_positive_definite <- is_positive_definite(gene_corrs_corrected)
  if (matrix_is_positive_definite) {
    IRdisplay::display("It worked!")
    IRdisplay::display(head(gene_corrs[1:10, 1:10]))
    IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))
    py_save_object(gene_corrs_corrected, OUTPUT_FILE)
  } else {
    eigenvalues <- eigen(gene_corrs_corrected)$values
    nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
    IRdisplay::display(length(nonpositive_eigenvalues))
    IRdisplay::display(nonpositive_eigenvalues)

    stop("Method failed to adjust matrix")
  }
}

[1] "Matrix is already positive definite!"

[1] TRUE

Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).